In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# unscaled, log-normalized counts, with conditions subsampled to the same number of cells 
# and 2000 highly variable genes calculated jointly across all perturbation conditions, including control, using scanpy28 with default parameters (Supplementary Methods)

In [3]:
from anndata import read_h5ad
import cupy as cp
from cupyx.scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import seaborn as sns
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
from sklearn.neighbors import LocalOutlierFactor
import string

import sys
sys.path.append("../../scxmatch/src")
from scxmatch import *

found cupy installation, will try use the GPU to calculate the distance matrix.


In [4]:
k = 100
#n = 1000

In [5]:
reference = 0.0
group_by = "dose_value"

In [ ]:
outlier_dfs = list()
data_path = "/data_nfs/datasets/scrnaseq_ji/"
datasets = ["sciplex_A549.hdf5", "sciplex_MCF7.hdf5", "sciplex_K562.hdf5"]
for i, dataset in enumerate(datasets): 
    adata = read_h5ad(os.path.join(data_path, dataset))    
    adata = scanpy_setup(adata)

    groups = adata.obs[group_by].unique()
    #reference_indices = list(np.random.choice(np.where(adata.obs[group_by] == reference)[0], size=n, replace=False))
    
    dfs = dict()
    for test_group in groups:
        if test_group == reference:
            continue
        
        #test_group_indices = np.where(adata.obs[group_by] == test_group)[0]
        #sampled_indices = list(np.random.choice(test_group_indices, size=n, replace=False))
        
        subset = adata[adata.obs[group_by].isin([reference, test_group])]
        (p, z, s), G, matching = rosenbaum(subset, group_by=group_by, reference=reference, test_group=test_group, k=k, return_matching=True)
        print(s)
        used_elements = sorted(list(chain.from_iterable(matching)))
        
        distances = cp.asnumpy(cdist(cp.array(subset.X.todense()), cp.array(subset.X.todense()), metric="sqeuclidean"))
        if np.min(distances) < 0:
            print("problem")
        
        lof = LocalOutlierFactor(n_neighbors=20, algorithm='auto', leaf_size=30, metric='precomputed')
        pred = lof.fit_predict(np.abs(distances))
        nof = lof.negative_outlier_factor_
        outliers = pd.DataFrame(nof, columns=["Negative outlier factor"])
        outliers["in matching"] = False
        outliers.iloc[used_elements, 1] = True
        dfs[test_group] = outliers
    df = pd.concat(dfs).reset_index().rename({"level_0": "Test group"}, axis=1)
    outlier_dfs.append(df)

XMatch_group
test         38945
reference     3772
Name: count, dtype: int64
calculating kNN graph.


/data/bionets/je30bery/anaconda3/envs/gt-292/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
p_vals = list()
for i, df in enumerate(outlier_dfs):
    for j, test_group in enumerate(df["Test group"].unique()):
        subset_true = df[(df["Test group"] == test_group) & (df["in matching"] == True)]["Negative outlier factor"]
        subset_false = df[(df["Test group"] == test_group) & (df["in matching"] == False)]["Negative outlier factor"]
        p_val = mannwhitneyu(subset_false, subset_true, alternative="less").pvalue
        p_vals.append(p_val)

In [ ]:
p_adjs = multipletests(p_vals, method="fdr_bh")[1]

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(12, 5), sharey=True)
labels = string.ascii_lowercase  
j_steps = int(len(p_adjs) / len(outlier_dfs))

for i, df in enumerate(outlier_dfs):
    legend = False if i != 1 else "full"
    sns.violinplot(df, y="Negative outlier factor", hue="in matching", x="Test group", cut=0, split=True, palette="magma", ax=axs[i], legend=legend) 
    axs[i].set_title(os.path.splitext(datasets[i])[0])
    
    for j in range(0, j_steps):
        axs[i].text(j, -1, f"${p_adjs[i * j_steps + j]:.3f}$", ha="center", va="bottom", fontsize=10, color="black")

    axs[i].text(
        -0.05, 1.05,  # Position (normalized figure coordinates)
        labels[i],   # Corresponding letter
        transform=axs[i].transAxes,  # Relative to subplot
        fontsize=10, fontweight='bold', va='top', ha='left'
    )

axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=2, title="Sample contained in matching with k=20")
plt.savefig("../plots/nof.pdf", bbox_inches="tight")